In [1]:
import copy
import weakref

### Variables are assigned to objects only after the objects are created

In [2]:
class Gizmo:
    def __init__(self):
        print("Gizmo id: {}".format(id(self)))

In [3]:
x = Gizmo()

Gizmo id: 1551333151456


In [4]:
# The second Gizmo is actually instantiated before the
# multiplication is attempted.
# y = Gizmo() * 10

In [5]:
dir()

['Gizmo',
 'In',
 'Out',
 '_',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_i4',
 '_i5',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'copy',
 'exit',
 'get_ipython',
 'quit',
 'weakref',
 'x']

But variable y was never created, because the exception happened while the righthand side of the assignment was being evaluated.

## Identity, Equality, and Aliases

In [6]:
charles = {'name': 'Charles L. Dodgson', 'born': 1832}
lewis = charles  # lewis is an alias for charles
lewis is charles

True

In [7]:
id(lewis) == id(charles)

True

In [8]:
lewis['balance'] = 950
lewis

{'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}

In [9]:
charles

{'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}

### alex and charles compare equal, but alex is not charles

In [10]:
alex = {'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}
alex == charles

True

In [11]:
alex is charles

False

In [12]:
id(alex), id(charles)

(1551333240640, 1551333155200)

## The Relative Immutability of Tuples

In [13]:
t1 = (1, 2, [30, 40])
t2 = (1, 2, [30, 40])

In [14]:
t1 == t2

True

In [15]:
t1 is t2

False

In [16]:
id(t1[-1])

1551333241280

In [17]:
t1[-1].append(99)

In [18]:
t1

(1, 2, [30, 40, 99])

In [19]:
id(t1[-1])  # The identity of t1[-1] has not changed, only its value.

1551333241280

In [20]:
t2

(1, 2, [30, 40])

In [21]:
t1 == t2

False

## Copies Are Shallow by Default

In [22]:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = list(l1)  # l2 is a shallow copy of l1.
l2

[3, [55, 44], (7, 8, 9)]

In [23]:
l2 == l1

True

In [24]:
l2 is l1

False

In [25]:
l1.append(100)  # Appending 100 to l1 has no effect on l2
l1[1].append(66)
l1[1].remove(55)  # Here we remove 55 from the inner list l1[1]. This affects l2 because l2[1] is
# bound to the same list as l1[1].
l1

[3, [44, 66], (7, 8, 9), 100]

In [26]:
l2

[3, [44, 66], (7, 8, 9)]

In [27]:
# For a mutable object like the list referred by l2[1], the operator += changes the
# list in place. This change is visible at l1[1], which is an alias for l2[1].
l2[1] += [33, 22]
# += on a tuple creates a new tuple and rebinds the variable l2[2] here. This is
# the same as doing l2[2] = l2[2] + (10, 11). Now the tuples in the last
# position of l1 and l2 are no longer the same object.
l2[2] += (10, 11)

In [28]:
l1

[3, [44, 66, 33, 22], (7, 8, 9), 100]

In [29]:
l2

[3, [44, 66, 33, 22], (7, 8, 9, 10, 11)]

## Deep and Shallow Copies of Arbitrary Objects

In [30]:
class Bus:
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
    def pick(self, name):
        self.passengers.append(name)
    def drop(self, name):
        self.passengers.remove(name)

### Effects of using copy versus deepcopy

In [31]:
bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)
id(bus1), id(bus2), id(bus3)

(1551333460240, 1551333460480, 1551333460624)

In [32]:
bus1.drop('Bill')
bus2.passengers

['Alice', 'Claire', 'David']

In [33]:
id(bus1.passengers), id(bus2.passengers), id(bus3.passengers)
# Inspection of the passengers atributes shows that bus1 and bus2 share the same
# list object, because bus2 is a shallow copy of bus1.

(1551333437312, 1551333437312, 1551333434944)

In [34]:
bus3.passengers
# bus3 is a deep copy of bus1, so its passengers attribute refers to another list.

['Alice', 'Bill', 'Claire', 'David']

### Cyclic references: b refers to a, and then is appended to a; deepcopy still manages to copy a

In [35]:
a = [10, 20]
b = [a, 30]
a.append(b)
a

[10, 20, [[...], 30]]

In [36]:
c = copy.deepcopy(a)
c

[10, 20, [[...], 30]]

## Function Parameters as References
***The only mode of parameter passing in Python is call by sharing***.

### A function may change any mutable object it receives

In [37]:
def f(a, b):
    a += b
    return a

In [38]:
x, y = 1, 2
f(x, y)

3

In [39]:
x, y

(1, 2)

In [40]:
a = [1, 2]
b = [3, 4]
f(a, b)

[1, 2, 3, 4]

In [41]:
a, b

([1, 2, 3, 4], [3, 4])

In [42]:
a = (1, 2)
b = (3, 4)
f(a, b)

(1, 2, 3, 4)

In [43]:
a, b

((1, 2), (3, 4))

## Mutable Types as Parameter Defaults: Bad Idea

In [44]:
class HauntedBus:
    """A bus model haunted by ghost passengers"""
    def __init__(self, passengers=[]):
        self.passengers = passengers
    def pick(self, name):
        self.passengers.append(name)
    def drop(self, name):
        self.passengers.remove(name)

In [45]:
bus1 = HauntedBus(['Alice', 'Bill'])

In [46]:
bus1.passengers

['Alice', 'Bill']

In [47]:
bus1.pick("Charlie")
bus1.drop("Alice")
bus1.passengers

['Bill', 'Charlie']

In [48]:
bus2 = HauntedBus()
bus2.pick('Carrie')
bus2.passengers

['Carrie']

In [49]:
bus3 = HauntedBus()
bus3.passengers

['Carrie']

In [50]:
bus3.pick('Dave')
bus2.passengers

['Carrie', 'Dave']

In [51]:
bus2.passengers is bus3.passengers

True

In [52]:
bus1.passengers

['Bill', 'Charlie']

The problem is that Bus instances that don’t get an initial passenger list end up sharing the same passenger list among themselves. When a HauntedBus is instantiated with passengers, it works as expected. Strange things happen only when a HauntedBus starts empty, because then self.passengers becomes an alias for the default value of the passengers parameter. ***The problem is that each default value is evaluated when the function is defined—i.e., usually when the module is loaded—and the default values become attributes of the function object***. So if a default value is a mutable object, and you change it, the change will affect every future call of the function.

In [53]:
dir(HauntedBus.__init__) # doctest: +ELLIPSIS

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [54]:
HauntedBus.__init__.__defaults__

(['Carrie', 'Dave'],)

In [55]:
HauntedBus.__init__.__defaults__[0] is bus2.passengers

True

## Defensive Programming with Mutable Parameters

### Passengers disappear when dropped by a TwilightBus

In [56]:
class TwilightBus:
    """A bus model that makes passengers vanish"""
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = passengers
    def pick(self, name):
        self.passengers.append(name)
    def drop(self, name):
        self.passengers.remove(name)

In [57]:
basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)
bus.drop('Tina')
bus.drop('Pat')
basketball_team

['Sue', 'Maya', 'Diana']

The assignment `self.passengers = passengers` makes `self.passengers`an alias for `passengers`, which is itself an alias for the actual argument passed to `__init__` (i.e., `basket
ball_team`).

When the methods `.remove()` and `.append()` are used with `self.passengers`, we are actually mutating the original list received as argument to the constructor.

The solution is to change the assignment to `self.passengers = list(passengers)`.

## del and Garbage Collection
### Watching the end of an object when no more references point to it

In [58]:
s1 = {1, 2, 3}
s2 = s1

In [59]:
def bye():
    print('Gone with the wind...')

In [60]:
ender = weakref.finalize(s1, bye)

In [61]:
ender.alive

True

In [62]:
del s1
ender.alive

True

In [63]:
# s2 still references {1,2,3}
s2 = 'spam'
ender.alive

Gone with the wind...


False

## Weak References
The presence of references is what keeps an object alive in memory. ***When the reference count of an object reaches zero, the garbage collector disposes of it***. But sometimes it is useful to have a reference to an object that does not keep it around longer than necessary. A common use case is a cache.

***Weak references to an object do not increase its reference count. The object that is the target of a reference is called the referent***. Therefore, we say that a weak reference does not prevent the referent from being garbage collected.

In [64]:
a_set = {0,1}
wref = weakref.ref(a_set)
wref

<weakref at 0x0000016932B014F0; to 'set' at 0x000001693294DF20>

In [65]:
wref()

{0, 1}

In [66]:
a_set = {2,3,4}
wref()

{0, 1}

In [67]:
wref() is None
# When this expression is evaluated, {0, 1} lives, therefore wref() is not None.
# But _ is then bound to the resulting value, False. Now there are no more strong
# references to {0, 1}.

False

In [68]:
wref() is None  # the result should be true
# Because the {0, 1} object is now gone, this last call to wref() returns None.

False

## The WeakValueDictionary Skit

In [69]:
class Cheese:
    def __init__(self, kind):
        self.kind = kind
    def __repr__(self):
        return 'Cheese(%r)' % self.kind

In [70]:
stock = weakref.WeakValueDictionary()
catalog = [Cheese('Red Leicester'), Cheese('Tilsit'),
    Cheese('Brie'), Cheese('Parmesan')]

In [71]:
for cheese in catalog:
    stock[cheese.kind] = cheese

In [72]:
sorted(stock.keys())

['Brie', 'Parmesan', 'Red Leicester', 'Tilsit']

In [73]:
del catalog
sorted(stock.keys())

['Parmesan']

In [74]:
del cheese
sorted(stock.keys())

[]

### A tuple built from another is actually the same exact tuple

In [75]:
t1 = (1, 2, 3)
t2 = tuple(t1)
print(t2 is t1)
t3 = t1[:]
print(t3 is t1)

True
True


### String literals may create shared objects

In [76]:
t1 = (1, 2, 3)
t3 = (1, 2, 3) #
print(t3 is t1) #
s1 = 'ABC'
s2 = 'ABC' #
print(s2 is s1)

False
True
